In [2]:
import requests

In [4]:

headers = {}
search_url = f"https://planner.dbcargo.com/stations.json"
params = {}
response = requests.get(search_url, headers=headers, params=params)

if response.status_code == 200:
    d = response.json()


[{'id': '000141', 'name': 'Sörnäinen', 'country': 10},
 {'id': '000406', 'name': 'Riihimäki', 'country': 10},
 {'id': '000471', 'name': 'Hämeenlinna', 'country': 10},
 {'id': '000737', 'name': 'Hanko', 'country': 10},
 {'id': '000752', 'name': 'Lappohja', 'country': 10},
 {'id': '000778', 'name': 'Mustola Satama', 'country': 10},
 {'id': '000794', 'name': 'Kirkniemi', 'country': 10},
 {'id': '001008', 'name': 'Lahti', 'country': 10},
 {'id': '001057', 'name': 'Uusikylä', 'country': 10},
 {'id': '001065', 'name': 'Loviisa Satama', 'country': 10},
 {'id': '001131', 'name': 'Heinola', 'country': 10},
 {'id': '001214', 'name': 'Uusikaupunki', 'country': 10},
 {'id': '001248', 'name': 'Naantali', 'country': 10},
 {'id': '001255', 'name': 'Raisio', 'country': 10},
 {'id': '001305', 'name': 'Turku', 'country': 10},
 {'id': '001354', 'name': 'Turku Satama', 'country': 10},
 {'id': '001503', 'name': 'Toijala', 'country': 10},
 {'id': '001537', 'name': 'Valkeakoski', 'country': 10},
 {'id': '001